In [3]:
# ==========================================
# langchain_101_clean.ipynb / langchain_101_clean.py
#
# 목표 (LangChain 101):
#   1) LLM(ChatGPT 같은 모델)을 LangChain에서 쓰는 방법
#   2) "Agent"가 무엇인지: LLM이 스스로 tool을 선택해서 쓰게 하는 컨트롤러
#   3) Wikipedia / Web search 같은 "Tools"를 agent에 붙이는 방법
#   4) (중요) API Key를 코드에 직접 쓰지 않는 안전한 패턴
#
# 이 코드는 "교육용 최소 예제"로, GitHub에 올려도
# secret scanning에 막히지 않도록 설계함.
# ==========================================


# -------------------------
# [0] 파이썬 표준 라이브러리 import
# -------------------------
import os
# os:
#   - 운영체제(OS) 환경변수(environment variable)를 읽고/쓰기 위해 사용
#   - OPENAI_API_KEY 같은 비밀키는 코드에 쓰지 말고 환경변수로 관리하는 것이 안전함


# -------------------------
# [1] LangChain에서 OpenAI Chat 모델을 쓰기 위한 import
# -------------------------
from langchain_openai import ChatOpenAI
# ChatOpenAI:
#   - OpenAI의 chat 모델(gpt-4o-mini 등)을 LangChain "LLM 인터페이스"로 감싸는 래퍼 클래스
#   - LangChain agent/chain에서 모델을 일관된 방식으로 호출할 수 있게 해줌


# -------------------------
# [2] LangChain v1 방식 agent 생성 함수 import
# -------------------------
from langchain.agents import create_agent
# create_agent:
#   - model(LLM) + tools(도구들) + system_prompt(행동 규칙)을 묶어서 "agent" 객체를 만들어줌
#   - agent는 "언제 tool을 쓸지"를 LLM에게 맡기는 orchestration layer(컨트롤러)


# -------------------------
# [3] 외부 정보 수집을 위한 Tool import
# -------------------------
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
# WikipediaQueryRun:
#   - 위키피디아를 검색/요약하는 tool
# DuckDuckGoSearchRun:
#   - DuckDuckGo로 웹 검색을 수행하는 tool (최신 정보에 유리)
# WikipediaAPIWrapper:
#   - WikipediaQueryRun 내부에서 실제 위키 API 호출을 돕는 wrapper
#
# 참고:
#   - langchain_community는 각종 tool/connector가 모여있는 패키지
#   - 핵심 패키지(langchain, langchain-core)와 분리되어 있음


# ==========================================================
# [4] API KEY 설정 (보안상 가장 중요)
# ==========================================================
# 절대 이렇게 하지 말 것:
#   os.environ["OPENAI_API_KEY"] = "sk-...."   # (X) 키가 코드/노트북/깃에 남아 사고남
#
# 권장:
#   터미널에서 환경변수로 설정하고 주피터를 실행
#     (macOS zsh 예시)
#       export OPENAI_API_KEY="sk-..."
#       jupyter lab
#
# 아래 assert는:
#   - 환경변수에 키가 없으면 바로 실행을 멈춰서
#     "키를 코드에 박아버리는" 사고를 예방함
assert os.getenv("OPENAI_API_KEY"), (
    "OPENAI_API_KEY is not set. "
    "Set it in your shell environment (e.g., export OPENAI_API_KEY='...') "
    "before running this notebook/script."
)



In [4]:
# ==========================================================
# [5] LLM(모델) 객체 생성
# ==========================================================
# ChatOpenAI는 실제로 OpenAI API를 호출하는 클라이언트 역할을 한다.
# - model="gpt-4o-mini": 빠르고 비용 효율적이며 tool 사용에 적합한 chat 모델
# - temperature=0: 랜덤성을 줄여서 재현성(reproducibility)을 높임 (교육/디버깅에 유리)
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

In [5]:
# ==========================================================
# [6] Tools(도구) 준비
# ==========================================================
# tools 리스트는 agent가 사용할 수 있는 "능력 목록"이다.
# agent는 사용자 질문을 보고:
#   - 그냥 LLM 지식으로 답할지
#   - Wikipedia tool을 쓸지
#   - Web search tool을 쓸지
# 를 스스로 결정한다.
#
# WikipediaQueryRun은 WikipediaAPIWrapper가 필요하므로 함께 생성한다.
tools = [
    WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()),
    DuckDuckGoSearchRun(),
]

In [6]:
# ==========================================================
# [7] Agent 생성
# ==========================================================
# create_agent에 전달하는 것들:
#   - model: LLM (생각/추론/결정 담당)
#   - tools: 외부 세계에서 정보를 가져오거나 행동을 수행하는 도구
#   - system_prompt: agent의 "기본 행동 규칙"
#
# system_prompt는 매우 중요:
#   - "외부 정보가 필요하면 tools를 써라" 같은 규칙을 명시하면,
#     agent가 환각 대신 검색을 더 잘 선택한다.
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=(
        "You are a helpful research assistant. "
        "Use tools when external information is needed. "
        "When you use tools, incorporate the results into a concise final answer."
    ),
)


In [7]:
# ==========================================================
# [8] Agent 실행 (invoke)
# ==========================================================
# agent.invoke(...)는 agent를 "한 번 실행"한다.
# 입력 형식은 보통 "messages" 리스트(대화 형태).
#
# messages는 OpenAI chat format과 유사하게:
#   - role: "user" / "assistant" / (tool 메시지 등)
#   - content: 텍스트
#
# agent 내부에서 일어나는 일(매우 중요):
#   1) LLM이 user 메시지를 읽음
#   2) "이 질문에 Wikipedia/Web search가 필요할까?"를 판단
#   3) 필요하면 해당 tool을 호출 (tool input 생성 → tool 실행)
#   4) tool 결과를 다시 LLM에게 제공
#   5) LLM이 최종 답변을 생성
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "What is NASA? Also, what is the latest update about the Artemis program? "
                    "Use Wikipedia for background and web search for recent updates."
                ),
            }
        ]
    }
)


/Users/sookim/miniconda3/envs/mcp/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/sookim/miniconda3/envs/mcp/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [8]:
# ==========================================================
# [9] 결과 출력
# ==========================================================
# result는 보통 dict 형태로 반환된다.
# 그 안에는 "messages"가 들어있고, 실행 중 생성된 메시지가 순서대로 담긴다.
#
# 예:
#   - user 메시지
#   - (필요시) tool call 관련 메시지
#   - tool 결과 메시지
#   - 최종 assistant 메시지
#
# 마지막 메시지가 보통 최종 답변이므로 [-1]을 사용한다.
# .content에는 "사람이 읽는 텍스트"만 들어있다.
print(result["messages"][-1].content)

### What is NASA?

The National Aeronautics and Space Administration (NASA) is an independent agency of the U.S. federal government responsible for the nation's civil space program and for research in aeronautics and space exploration. Established in 1958, NASA succeeded the National Advisory Committee for Aeronautics (NACA) to focus on peaceful applications in space science. Headquartered in Washington, D.C., NASA operates ten field centers across the U.S. and is organized into various mission directorates, including Science, Space Operations, and Exploration Systems Development.

NASA has led many significant space exploration programs, including the Apollo missions, Skylab, and the Space Shuttle. It also supports the International Space Station (ISS) and oversees the development of the Orion spacecraft and the Space Launch System for the Artemis program. The agency's budget for FY2022 was approximately $24.041 billion, and it employs around 18,400 people.

### Latest Update on the A